In [1]:
# 1. Install necessary libraries
# We use 'quiet' to keep the output clean, remove it if you want to see installation details
!pip install pandas nltk textblob --quiet

# 2. Import modules
import pandas as pd
import random
import json
from datetime import datetime
import nltk
from textblob import TextBlob # Good for simple sentiment detection (keeping mood positive)

# 3. Download required NLTK data for tokenizing text
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True) # Useful corpus for reference

# 4. Define a simple configuration for the bot
BOT_CONFIG = {
    "bot_name": "Remi", # Short for Reminiscence
    "user_name": "User", # We can change this dynamically later
    "session_start": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

print(f"✅ Environment Ready for {BOT_CONFIG['bot_name']}!")
print(f"Session started at: {BOT_CONFIG['session_start']}")

✅ Environment Ready for Remi!
Session started at: 2026-02-08 22:35:44


From the code above, 

pandas: We will use this to save conversation logs (very important for analysis later).

textblob: A simple library to detect sentiment. If a user gets agitated (negative sentiment), we can program the bot to change the subject.

BOT_CONFIG: A dictionary to store basic settings we can reference later.

We now want to move to Step 2: Defining the "Reminiscence Knowledge Base" This is the brain the bot will draw from.

In [2]:
# Define the topics and specific reminiscence prompts
reminiscence_topics = {
    "childhood": [
        "What was your favorite game to play outside when you were young?",
        "Do you remember the house you grew up in? What was your favorite room?",
        "What was your favorite home-cooked meal as a child?",
        "Did you have any pets growing up? What were their names?"
    ],
    "career": [
        "What was your very first job? Did you enjoy it?",
        "What is a professional accomplishment you are most proud of?",
        "Who was the best boss or mentor you ever had?",
        "How did your work change over the years?"
    ],
    "hobbies": [
        "What hobbies have you enjoyed throughout your life?",
        "Is there a particular song or type of music that always makes you smile?",
        "Did you enjoy traveling? Where was your favorite place to visit?",
        "What is a book or movie that had a big impact on you?"
    ],
    "family": [
        "How did you meet your spouse or partner?",
        "What is a favorite holiday tradition your family had?",
        "Tell me about a memorable family vacation.",
        "What values were most important to your parents?"
    ]
}

# Simple function to get a random question from a topic
def get_prompt(topic):
    if topic in reminiscence_topics:
        return random.choice(reminiscence_topics[topic])
    else:
        return "I'd love to hear more about that. Can you tell me more?"

print("✅ Knowledge Base Loaded with topics:", list(reminiscence_topics.keys()))

✅ Knowledge Base Loaded with topics: ['childhood', 'career', 'hobbies', 'family']


We are keeping the conversation focused on therapeutic areas and later we can add other information like music, school, e.t.c.

Defining The Logic Functions:

Next, we are going to build the "Brain" It is a function that takes what the user says, analyzes it and decides the memory topic to trigger next.

In [7]:
# --- REMI V4: The "Respectful" Update ---

chat_state = {
    "current_topic": None,
    "asked_questions": set(),
    "standing_by": False  # New mode: If True, Remi stops asking questions
}

def get_respectful_response(user_input):
    text = user_input.lower()
    
    # 1. CHECK IF USER WANTS TO RESUME
    if chat_state["standing_by"]:
        if any(word in text for word in ["hello", "hi", "ready", "ok", "talk"]):
            chat_state["standing_by"] = False
            return "Welcome back. I'm listening."
        else:
            return "..." # Remi stays silent/passive

    # 2. CHECK FOR "STOP" / HOSTILITY
    stop_triggers = ["stop", "shut up", "quiet", "don't ask", "leave me alone", "go to hell", "hate you"]
    if any(trigger in text for trigger in stop_triggers):
        chat_state["standing_by"] = True
        return "I hear you. I will stop asking questions now. I'll be here quietly if you want to chat later. (Type 'hello' to resume)"

    # 3. CHECK FOR REFUSAL ("I won't say", "No")
    refusal_triggers = ["no", "won't", "don't want", "refuse", "skip", "pass"]
    if any(trigger in text for trigger in refusal_triggers):
        # Switch topic immediately without scolding
        new_topic = random.choice(list(reminiscence_topics.keys()))
        chat_state["current_topic"] = new_topic
        question = get_new_prompt(new_topic)
        return f"Understood. We can skip that. How about {new_topic}? {question}"

    # 4. NORMAL CONVERSATION FLOW
    # (Same logic as before, but safer)
    detected = detect_topic(user_input)
    if detected:
        chat_state["current_topic"] = detected
        question = get_new_prompt(detected)
        return f"Ah, {detected}. {question}"
    
    # If we are in a topic, keep going
    if chat_state["current_topic"]:
        question = get_new_prompt(chat_state["current_topic"])
        if not question: # If out of questions
             return "I've run out of questions on this topic! Tell me anything else on your mind."
        return f"{random.choice(acknowledgments)} {question}"

    # Fallback
    new_topic = random.choice(list(reminiscence_topics.keys()))
    chat_state["current_topic"] = new_topic
    return f"Let's talk about {new_topic}. {get_new_prompt(new_topic)}"

print("✅ Remi V4 Logic Loaded: Includes 'Stand By' mode.")

✅ Remi V4 Logic Loaded: Includes 'Stand By' mode.


In [ ]:
print("--- Remi V4 (Respectful Mode) ---")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Remi: Goodbye.")
        break
        
    response = get_respectful_response(user_input)
    print(f"Remi: {response}")

--- Remi V4 (Respectful Mode) ---


You:  do for dog


Remi: Let's talk about career. What is a professional accomplishment you are most proud of?


You:  why not start by greeting me


Remi: Understood. We can skip that. How about childhood? What was your favorite home-cooked meal as a child?


...

Remi: Understood. We can skip that. How about family? What is a favorite holiday tradition your family had?


You:  I don't have


Remi: I see. How did you meet your spouse or partner?


You:  no spouse


Remi: Understood. We can skip that. How about hobbies? Is there a particular song or type of music that always makes you smile?


The above is important because of context awareness, i.e if the user says I loved my mum, the bot will detect family and ask a family question next. It acts as a safety valve i.e < -0.5 check if the patient gets upset, the bot tries to pivot to a happier subject(music/hobbies)

we are now going to stitch all those pieces together into a running loop. This will be the main chat loop.